In [35]:
%pip install albumentations opencv-python
%pip install torchvision--(datasets version 0.3.2 , transforms version 0.3.2)

Note: you may need to restart the kernel to use updated packages.
/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `/home/codespace/.python/current/bin/python -m pip install torchvision--(datasets version 0.3.2 , transforms version 0.3.2)'
Note: you may need to restart the kernel to use updated packages.


In [15]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import cv2
import numpy as np
from spectral import envi, principal_components
from tqdm import tqdm
import random
import albumentations as A


In [16]:
def append_byte_order(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    if not any("byte order" in line.lower() for line in lines):
        with open(file_path, 'a') as file:
            file.write("\nbyte order = 0\n")

def generate_false_color(img_path, hdr_path):
    append_byte_order(hdr_path)
    img = envi.open(hdr_path, img_path)
    cube = img.load()
    pca = principal_components(cube)
    pc1 = pca.transform(cube)[:, :, 0]
    pc1_norm = 255 * (pc1 - pc1.min()) / (pc1.max() - pc1.min())
    pc1_norm = pc1_norm.astype(np.uint8)
    return cv2.applyColorMap(pc1_norm, cv2.COLORMAP_JET)

def convert_and_save_images(root_dir, output_dir):
    classes = ['serie healthy leaves', 'serie infected leaves']
    os.makedirs(output_dir, exist_ok=True)

    for cls in classes:
        input_cls_path = os.path.join(root_dir, cls)
        output_cls_path = os.path.join(output_dir, cls)
        os.makedirs(output_cls_path, exist_ok=True)

        files = os.listdir(input_cls_path)
        hdr_files = [f for f in files if f.endswith('.hdr')]

        for hdr_file in tqdm(hdr_files, desc=f'Processing {cls}'):
            base = os.path.splitext(hdr_file)[0]
            img_file = base + '.img'
            hdr_path = os.path.join(input_cls_path, hdr_file)
            img_path = os.path.join(input_cls_path, img_file)

            if not os.path.exists(img_path):
                continue

            try:
                vis_img = generate_false_color(img_path, hdr_path)
                save_path = os.path.join(output_cls_path, base + '.png')
                cv2.imwrite(save_path, vis_img)
            except Exception as e:
                print(f"Error with {hdr_file}: {e}")

In [17]:
convert_and_save_images(
    root_dir='DataFiles',
    output_dir='processed_data'
)

Processing serie infected leaves: 100%|██████████| 7/7 [00:02<00:00,  2.75it/s]


In [18]:
# === CONFIGURATION HEALTHY IMAGES===
input_folder_1 = "processed_data/serie healthy leaves"       # Change this to your folder path
output_folder_1 = "aug_processed_data/serie_healthy_leaves_augmented"          # Where to save augmented images
total_augmented = 100

In [19]:
# === CONFIGURATION HEALTHY IMAGES===
input_folder = "processed_data/serie infected leaves"       # Change this to your folder path
output_folder = "aug_processed_data/series_infected_leaves_augmented"          # Where to save augmented images
total_augmented = 100

In [20]:
# === AUGMENTATION PIPELINE ===
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=25, p=0.7),
    A.RandomBrightnessContrast(p=0.5),
    A.GaussianBlur(p=0.3),
])

# === LOAD IMAGES ===
def load_images(folder):
    images, filenames = [], []
    for filename in os.listdir(folder):
        if filename.lower().endswith(".png"):
            path = os.path.join(folder, filename)
            img = cv2.imread(path)
            if img is not None:
                images.append(img)
                filenames.append(filename)
    return images, filenames

# === AUGMENTATION FUNCTION ===
def augment_images(input_folder, output_folder, total_augmented):
    os.makedirs(output_folder, exist_ok=True)

    images, filenames = load_images(input_folder)
    n = len(images)

    if n == 0:
        print("❌ No .png images found in the input folder.")
        return

    print(f"📂 Found {n} images. Generating {total_augmented} augmented images...")

    for i in tqdm(range(total_augmented), desc="Augmenting"):
        idx = random.randint(0, n - 1)
        image = images[idx]

        # Apply augmentation WITHOUT ToTensorV2 and Normalize for saving
        augmented = A.Compose([
            A.Resize(128, 128),
            # Use the rest from above but exclude Normalize and ToTensorV2
            A.OneOf([
                A.HorizontalFlip(p=0.5),
                A.VerticalFlip(p=0.5),
                A.RandomRotate90(p=0.5),
            ], p=0.7),

            A.ShiftScaleRotate(
                shift_limit=0.1, scale_limit=0.2, rotate_limit=45, p=0.7
            ),

            A.OneOf([
                A.GaussianBlur(blur_limit=3, p=0.3),
                A.MotionBlur(blur_limit=3, p=0.3),
                A.MedianBlur(blur_limit=3, p=0.3),
            ], p=0.3),

            A.OneOf([
                A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.7),
                A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.7),
                A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.7),
            ], p=0.7),

            A.CoarseDropout(max_holes=8, max_height=16, max_width=16, p=0.5),

            A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
        ])(image=image)["image"]

        new_filename = f"aug_{i}_{filenames[idx]}"
        save_path = os.path.join(output_folder, new_filename)
        cv2.imwrite(save_path, augmented)

    print(f"\n✅ Done! Augmented images saved in: {output_folder}")

# === RUN ===
augment_images(input_folder, output_folder, total_augmented)
augment_images(input_folder_1, output_folder_1, total_augmented)

📂 Found 7 images. Generating 100 augmented images...


Augmenting:   0%|          | 0/100 [00:00<?, ?it/s]/usr/local/python/3.12.1/lib/python3.12/site-packages/albumentations/core/validation.py:111: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/tmp/ipykernel_45177/880064550.py:64: UserWarning: Argument(s) 'max_holes, max_height, max_width' are not valid for transform CoarseDropout
  A.CoarseDropout(max_holes=8, max_height=16, max_width=16, p=0.5),
/tmp/ipykernel_45177/880064550.py:66: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
Augmenting:   8%|▊         | 8/100 [00:00<00:01, 73.07it/s]

Augmenting: 100%|██████████| 100/100 [00:00<00:00, 145.30it/s]



✅ Done! Augmented images saved in: aug_processed_data/series_infected_leaves_augmented
📂 Found 7 images. Generating 100 augmented images...


Augmenting: 100%|██████████| 100/100 [00:00<00:00, 178.06it/s]


✅ Done! Augmented images saved in: aug_processed_data/serie_healthy_leaves_augmented
